# Demonstration of low-level functions within MPChecker2
 - This is likely to mainly be of use while developing the code 
 - 20200122
 - MJP

In [5]:
# Standard imports 
import os , sys
import numpy as np 
import random
import string
import importlib
import json 
import glob
import shutil

# orbit_checby imports 
sys.path.append( "/Users/matthewjohnpayne/Envs/orbit_cheby/orbit_cheby/" )
import nbody_reader
import orbit_cheby
importlib.reload(nbody_reader)
importlib.reload(orbit_cheby)

# mpchecker2 imports 
sys.path.append(os.path.join(os.path.split(os.getcwd())[0], 'mpchecker2'))
import sql
import precalc
importlib.reload(sql)
importlib.reload(precalc)


/Users/matthewjohnpayne/Envs/mpcvenv/mpchecker2/mpchecker2
/Users/matthewjohnpayne/Envs/mpcvenv/mpchecker2/mpchecker2


<module 'precalc' from '/Users/matthewjohnpayne/Envs/mpcvenv/mpchecker2/mpchecker2/precalc.py'>

# Background
 - I expect that the "orbit_cheby" module exists and has acted to take the results of an n-body integration and convert them into a dictionary of chebyshev coefficients (with seprate coefficient sets for each "sector" of time)  - There is a single dictionary for each object


# --------- Demo-Data Functions ---------------------

 - These functions create fake data for the sake of demonstrating functionalities

In [6]:
def create_sample_cheby_dict():
    importlib.reload(nbody_reader)
    importlib.reload(orbit_cheby)

    # Set up a filepath (to be created)
    text_filepath = os.path.join(os.path.dirname(os.getcwd() ), 'dev_data', '2022AA_demo.txt')

    # Use convenience func in nbody_reader to create a text file 
    nbody_reader.create_nbody_txt(text_filepath)
    assert os.path.isfile(text_filepath)

    # Create cheby dict 
    multi_sector_cheby_dict = orbit_cheby.generate_multi_sector_cheby_dict_from_nbody_text( 
                                        text_filepath ,
                                        40001,
                                        59999, 
                                        minorder=17,
                                        CHECK = True )

    return multi_sector_cheby_dict

In [7]:
multi_sector_cheby_dict = create_sample_cheby_dict()

for key in multi_sector_cheby_dict.keys():
    if key == 'sectors':
        print(key,  ' ; Number of sectors = ' , len(multi_sector_cheby_dict['sectors']) )
    else:
        print(key, multi_sector_cheby_dict[key])

/Users/matthewjohnpayne/Envs/orbit_cheby/orbit_cheby/orbit_cheby.py:386: RankWarning: The fit may be poorly conditioned
  candidateCoeffs = np.polynomial.chebyshev.chebfit(t, arr, int(np.ceil(order)) )


unpacked_designation 2022 AA
MJP_TDB_final 59999
MJP_TDB_init 40001
sectors  ; Number of sectors =  625


# --------- SQL ---------------------
 - I was doing a lot of reinventing the wheel 
 - I am going to try to radically simplify things and just use a/an SQLITE database to store the required data
 
Required Structure for db ... 
 - Cheby Coefficient table organized with one-row-per-object, and with individual fields (columns) for each 32-day time-sector
 - I assume that the contents of each "cell" will have to be a "blob" containing all required coeffs for all required coords/covars
 - There should also be a column (or separate table?) recording the designation-string

In [41]:
# --- Demonstrate some convenience code to make database with required tables ---
# -------------------------------------------------------------------------------

importlib.reload(sql)
importlib.reload(precalc)

def convenience_func_create_db_and_tables():
    B = precalc.Base() 
    
    # In order to save data, we require sql-db to exist, so let's set that up...
    # Force deletion then creation of db...
    if os.path.isfile( sql.fetch_db_filepath() ):
        os.remove(sql.fetch_db_filepath())
    conn = sql.create_connection( sql.fetch_db_filepath() )
    cur  = conn.cursor()
    
    # Test creation of db
    assert os.path.isfile( os.path.join( sql.fetch_db_filepath() ) ), 'no db'

    # Create required table(s)
    sql.create_specific_tables(conn)

    # Double-check that this worked by getting the count of tables with the name
    # - if the count is 1, then table exists
    cur.execute('SELECT name from sqlite_master WHERE type = "table" AND name = "objects_by_jdhp"')
    assert len(cur.fetchone()) == 1 , 'jdhp table does not exist'
    cur.execute('SELECT name from sqlite_master WHERE type = "table" AND name = "object_coefficients"')
    assert len(cur.fetchone()) == 1 , 'coeff table does not exist'

    conn.close()

# Create db & tables
convenience_func_create_db_and_tables()
# Repeat check that db exists (other checks done within *convenience_func_create_db_and_tables()* )
os.path.isfile( os.path.join( sql.fetch_db_filepath() ) )

/Users/matthewjohnpayne/Envs/mpcvenv/mpchecker2/mpchecker2


True

In [48]:
# --- Demonstrate code to "upsert" dictionary into mpchecker2 sqlite db ---------
# -------------------------------------------------------------------------------
importlib.reload(sql)
importlib.reload(precalc)

# (1) Low level direct call using sql
conn = sql.create_connection( sql.fetch_db_filepath() )
#sql.upsert_multi_sector_cheby_dict(conn, multi_sector_cheby_dict)
print( 
    orbit_cheby.get_valid_range_of_dates_from_cheby(multi_sector_cheby_dict) 
)

# (2) Higher level call using PreCalc() instance 
#P = precalc.PreCalc() 


/Users/matthewjohnpayne/Envs/mpcvenv/mpchecker2/mpchecker2
(40001, 59999)


In [50]:
multi_sector_cheby_dict['sectors'][0]

{'MJP_TDB_final': 40033,
 'MJP_TDB_init': 40001,
 'coeffs': array([[ 2.31523079e+05, -1.57934829e+05,  0.00000000e+00,
          0.00000000e+00,  2.31523079e+04,  4.63046159e+04,
          0.00000000e+00,  2.31523079e-04,  4.63046158e-04,
          6.94569237e-04,  9.26092317e-04,  1.15761540e-03,
          1.38913847e-03,  1.62066155e-03,  1.85218463e-03,
          2.08370771e-03,  2.31523079e-03,  2.54675387e-03,
          2.77827695e-03,  3.00980003e-03,  3.24132311e-03,
          3.47284618e-03,  3.70436927e-03,  3.93589235e-03,
          4.16741543e-03,  4.39893850e-03,  4.63046158e-03],
        [-9.58620190e-01,  7.84009137e-01,  0.00000000e+00,
          0.00000000e+00, -9.58620191e-02, -1.91724038e-01,
          0.00000000e+00, -9.58620191e-10, -1.91724038e-09,
         -2.87586057e-09, -3.83448076e-09, -4.79310095e-09,
         -5.75172114e-09, -6.71034133e-09, -7.66896152e-09,
         -8.62758172e-09, -9.58620190e-09, -1.05448221e-08,
         -1.15034423e-08, -1.24620625e-0

In [89]:
importlib.reload(orbit_cheby)
sector_length_days = 32
mindate,maxdate = 2450000, 2456000
sector_numbers, sector_JD0s = orbit_cheby.map_JD_to_sector_number_and_sector_start_JD( [mindate,maxdate] )
print(sector_numbers)
print(sector_JD0s)
sector_numbers[0]           = sector_numbers[0]  if mindate - sector_JD0s[0]                        < 1 else sector_numbers[0]  + 1
print('...', sector_JD0s[-1] + orbit_cheby.sector_length_days , maxdate, sector_JD0s[-1] + orbit_cheby.sector_length_days - maxdate)
sector_numbers[-1]          = sector_numbers[-1] if sector_JD0s[-1] + orbit_cheby.sector_length_days - maxdate  < 1 else sector_numbers[-1] - 1
print(sector_numbers)


[312 500]
[2449984 2456000]
... 2456032 2456000 32
[313 499]


# --------- Lower-Level Data Functions ---------------------


### --------- Base Class ---------------------
 - 20200122
 - Illustrate the usage of the Base Class
 - This is intended to be a parent class to hold some file/directory definitions & methods

In [72]:
# Create an instance
B = precalc.Base()

# Illustrate some available variables
expected_var_names = ['Names', 'Coeffs', 'HP', 'HP_nside', 'HP_order', 'hp', 'npix']
print('illustrating variables ...')
for var_name in expected_var_names:
    value = B.__dict__[var_name]
    print( "%20s:%20s" % (var_name, value) )

    
# Check that number of pixels for the healpix is as expected 
print('\nChecking number of pixels')
assert B.npix == 12 * (B.HP_nside)**2 , ' if see message: number of pixels is wrong' 
print( B.npix, B.HP_nside , 12 * (B.HP_nside)**2) 

# Look at the name of the directory that is used to store data 
#  - check that the directory exists 
#  - (calling '_fetch_data_directory' causes the directory to be created if it does *not* exist already)
print("\nLooking at the save-directory")
print(B._fetch_data_directory(), " : exists =", os.path.isdir(B._fetch_data_directory()))

illustrating variables ...
               Names:           names.txt
              Coeffs:   coefficients.json
                  HP:        _healpix.txt
            HP_nside:                  16
            HP_order:              nested
                  hp:<astropy_healpix.high_level.HEALPix object at 0x11e9b25c0>
                npix:                3072

Checking number of pixels
3072 16 3072

Looking at the save-directory
/Users/matthewjohnpayne/.mpchecker_data  : exists = True


### --------- Names Class ---------------------
 - 20200122
 - Illustrate the usage of the Names Class
 - This is intended to handle ...
        List of names/designation (strings) 
        Allows implicit mapping of names <--> integers (posn in list)
        Allows easier lookups later-on
        
        Methods
        -------
        
        insert()
        get_number()
        get_name() 
        
        _read_file()
        _write_file()
        _get_filepath()
        


##### Demonstrating initialization of the Names class ...

In [73]:
N = precalc.Names() 

# Names inherits the Base methods
print('\nDemonstrate that Names-class inherits the Base (parent) methods')
print(N.Names , N._fetch_data_directory())

# File where name data will be saved ... 
print('\nDemonstrate : File where name data will be saved')
print(N._get_names_filepath())

# For the sake of demo, ensure name-data file does not exist
print('\nFor the sake of demo, ensure name-data file does not exist')
if os.path.isfile(N._get_names_filepath()):
    os.remove(N._get_names_filepath())
    
# At this point the underlying file is empty / does not exist ...
# ... and so is the names_dict which holds the data
print("\nDemonstrate : names_dict is empty")
print(N.names_dict)


Demonstrate that Names-class inherits the Base (parent) methods
names.txt /Users/matthewjohnpayne/.mpchecker_data

Demonstrate : File where name data will be saved
/Users/matthewjohnpayne/.mpchecker_data/names.txt

For the sake of demo, ensure name-data file does not exist

Demonstrate : names_dict is empty
OrderedDict([('1997 CU26', 0)])


##### Now lets use the sample data created above to start to populate the Names class ...

In [74]:
# define a demo name
name=demo_name_dict[10199]

# insert the name 
print('Inserting demo name : ' , name )
i = N.insert(name)
print("The returned value from insert is the position/index of the name: ", i) 
print('The names-dictionary is now populated:\n\t', N.names_dict, '\n')

# attempt to insert the same name again 
i = N.insert(name)
print("The returned value from insert is the *same* position/index of the name: ", i) 
print("The names-dictionary is unchanged", N.names_dict, '\n')

# write the data to file 
N._write_names_file()

# check that the file now exists and has content
print("Demo: the file [%s] now exists and has the following content ... " % N._get_names_filepath())
assert os.path.isfile(N._get_names_filepath())
with open(N._get_names_filepath(), 'r') as fp:
    print(fp.readlines())

Inserting demo name :  1997 CU26
The returned value from insert is the position/index of the name:  0
The names-dictionary is now populated:
	 OrderedDict([('1997 CU26', 0)]) 

The returned value from insert is the *same* position/index of the name:  0
The names-dictionary is unchanged OrderedDict([('1997 CU26', 0)]) 

Demo: the file [/Users/matthewjohnpayne/.mpchecker_data/names.txt] now exists and has the following content ... 
['1997 CU26\n']


##### Now that the above example has populated the names-file on disk, we can read from it

In [85]:
# Note that here we are decalring a new Names instance:
# - By default it reads the file-from-disk and populates a names_dict variable 
N = precalc.Names() 
print('After re-initialize: names-dict populated as file is read by default:\n\t', N.names_dict, '\n')


After re-initialize: names-dict populated as file is read by default:
	 OrderedDict([('1997 CU26', 0)]) 



##### Demonstrate the use of the main public "get_number" and "get_name" routines

In [86]:
# Get the number for a name-string 
print('Demo: Getting the number for a known string-name ...')
name = '1997 CU26'
print(
    name,
    " : ", 
    N.get_number(name)
)

# Get the name-string for an integer-index
print('Demo: Getting the name for a given index-number ...')
number = 0 
print( number , ' : ' , N.get_name(number) )

Demo: Getting the number for a known string-name ...
1997 CU26  :  0
Demo: Getting the name for a given index-number ...
0  :  1997 CU26


### --------- HPData Class ---------------------
 - 20200122
 - Illustrate some of the variables and methods that are available within the "HPData" class

        Precalculated HP location of objects
        - Probably from the geocenter
        - Probably on a nightly basis
        
        
        Methods
        -------


In [190]:
importlib.reload(precalc)

# Instantiate 
H = precalc.HPData()

# Demonstrate that HPData inherits the Base methods
print('\nDemonstrate that HPData-class inherits the Base (parent) methods')
print(H.Names , H._fetch_data_directory())



Demonstrate that HPData-class inherits the Base (parent) methods
names.txt /Users/matthewjohnpayne/.mpchecker_data


##### Demonstrate initialization of HP file structure 
 - When we want to quickl ysearch for HP, we want to always be able to look into files of a given, gauranteed structure 
 - So we probably need to coherently initialize the HP file structure
 - This takes a minute or two 

In [199]:
importlib.reload(precalc)

# Instantiate 
H = precalc.HPData()

# Delete the entire saved-data directory to ensure completely clean ...
shutil.rmtree(P._fetch_data_directory())

# _create_all_required_empty_block_files
H._create_all_required_empty_block_files()

# Demonstrate that a lot of empty files now exist with the expected names ...
files = glob.glob( P._fetch_data_directory() + '/HP*dat')
print('Total number of files created = len(files) = ', len(files))
print('\t files[0] = ', files[0])

Total number of files created = len(files) =  201
	 files[0] =  /Users/matthewjohnpayne/.mpchecker_data/HP_2452500.dat


#### Demonstrate the write/save of some new HP Data ...

In [188]:
importlib.reload(precalc)
H = precalc.HPData()

# Here we imagine that some functionality 
# (to be demonstrated in the *ObjectCoefficients* section below) 
# has calculated the HP-locations of an object on each 
# of a range of integer days
JDlist = np.arange(2458000,2458010,1)
HPlist = [457, 86, 2089, 2262, 22, 429, 96, 2698, 2374, 300]

# Demonstrate: Save HP data to file 
# N.B.(1) *update_object_HP* wants to know *old_HPlist*, a list of old HP locations 
#  - This is to allow easy replacement
#  - If new, then input this as [None]*len(JDlist)
#  - This will be handled automatically by calls to *update_object_HP()* from *PreCalc.upsert()*
# N.B.(2) This demands knowledge of the 'number', i.e. the unique object-number to be added to the list(s)
# - This is/will-be handled automatically in practice, but for now I do it by hand by setting number=0
number = 0 
oldHPlist = [None]*len(JDlist)
H.update_object_HP(number, JDlist, oldHPlist, HPlist)

NameError: name 'HP' is not defined

### --------- ObjectCoefficients Class ---------------------
 - 20200122
 - Illustrate some of the variables and methods that are available within the "ObjectCoefficients" class
 
        Fitted coefficients for each object 
        
        Basic structure is a list of dictionaries
        
        WIP to define what these will be
         - At least positional information 
         - Probably velocity information
         - Some kind of Uncert/Covar information
         
        
        Methods
        -------
        
        


In [167]:
# Reload to ensure have latest version while developing
importlib.reload(precalc)

# Delete the entire saved-data directory to ensure completely clean ...
shutil.rmtree(P._fetch_data_directory())

# Initialize an ObjectCoefficients object  
OC = precalc.ObjectCoefficients()

# Look at the 'coeff_list' variable: it should be an empty list at this point
print('Empty coeff-list:\n\t', OC.coeff_list)

# Note the OC has access to the names_dict from the Names object
print('Names dict is available:\n\t', OC.names_dict)

# Now insert one of the sets of coefficients from the demo_coeff_dict 
# - Note that it demands knowledge of the 'number', i.e. the position in the list
# - This is/will-be handled automatically in practice, but for now I do it by hand be setting number=0
name = demo_name_dict[1944]
new_coeff_dict = demo_coeff_dict[name]
number = 0 
OC.upsert_coeff( number,  new_coeff_dict )
print('After upcert, co-effs populated:\n\t len(OC.coeff_list)=',len(OC.coeff_list) )
 
# For the sake of demo, ensure coeff-data file does not exist
if os.path.isfile(OC._get_coeff_filepath()):
    os.remove(OC._get_coeff_filepath())

# write the data to file **NOT WRITING PROPERLY ***
OC._write_coeff_file()

# check that the file now exists and has content
assert os.path.isfile(OC._get_coeff_filepath())
print('file has content:\n\t len(OC._read_coeff_file())= ', len(OC._read_coeff_file()) )
print('\t type(OC._read_coeff_file()) = ', type(OC._read_coeff_file()))
print('\t type(OC._read_coeff_file()[0]) = ', type(OC._read_coeff_file()[0]))


Empty coeff-list:
	 []
Names dict is available:
	 OrderedDict()
After upcert, co-effs populated:
	 len(OC.coeff_list)= 1
file has content:
	 len(OC._read_coeff_file())=  1
	 type(OC._read_coeff_file()) =  <class 'list'>
	 type(OC._read_coeff_file()[0]) =  <class 'dict'>


##### Now that the coefficient file has been populated, it will be read directly when a coefficient object is declared:

In [175]:
importlib.reload(precalc)

# Initialize an ObjectCoefficients object  
OC = precalc.ObjectCoefficients()

# Look at the 'coeff_list' variable: it should be an populated now (because of the above cell)
print('coeff-list...')#, OC.coeff_list)
print('\t type(OC.coeff_list) = ', type(OC.coeff_list) )
print('\t type(OC.coeff_list[0]) = ', type(OC.coeff_list[0]))
print('\t OC.coeff_list[0]["x"] = ', OC.coeff_list[0]["x"])


coeff-list...
	 type(OC.coeff_list) =  <class 'list'>
	 type(OC.coeff_list[0]) =  <class 'dict'>
	 OC.coeff_list[0]["x"] =  {'55006.0': [29140179.77107003, -177.16929527916486, -0.0039007738371499086, -3.214923426327295e-08, -1.4564994463506918e-13, 2.976075553550126e-19, 1.5557048366813042e-23, 2.0597374072143643e-28, 1.8760880964657482e-33, 1.1623971883438532e-38, 1.8825825337154302e-44, -7.521390605230102e-49, -1.3636902772606969e-53, -1.3543247639104823e-58, -5.1161287533508565e-64, 1.3876405625503416e-68], '50862.0': [-918912.4526366527, 6.040067257807393, 0.00014378407624141823, 1.283601517734911e-09, 6.3275930428921504e-15, -1.4340220730105496e-20, -7.8295028685022885e-25, -1.1258027276361841e-29, -1.1054760830078196e-34, -7.408685012790916e-40, -1.2909471994744592e-45, 5.605943016780568e-50, 1.0950069727158768e-54, 1.1822642154158095e-59, 4.784234491828354e-65, -1.409079176868795e-69], '44454.0': [-395435.8643960237, 2.888782921124685, 8.011157140033333e-05, 8.243575230942093e-

##### Demonstration of code to read & write HEALPIX coefficients 

def _generate_HP_from_CoeffDict(self, JDlist, coeff_dict):

        '''
            This function calculates the HP location of a SINGLE-OBJECT over a series of MANY dates
            
            inputs:
            -------
            JDlist: list of integers
             - list of integer Julian Days for which calculations should be performed
             
            coeff_dict: dictionary 
             - generalized coefficient dictionary 
             - must contain sufficient information to allow HP to be extracted/calculated
             
            returns:
            --------
            HPlist: list
             - will be list of integers if coeff-dict properly specified
             - in the event of improper specificiation and/or coeff_dict == None, then a list of "None" will be returned
             - len(HPlist) == len(JDlist)
         '''


In [184]:
importlib.reload(precalc)

# Initialize an ObjectCoefficients object  
OC = precalc.ObjectCoefficients()

# Look at the first coeff-dict in the 'coeff_list': 
# - use this as an imput to the "_generate_HP_from_CoeffDict" function
coeff_dict = OC.coeff_list[0]
JDlist = np.arange(2458000,2458010,1)
HPlist = OC._generate_HP_from_CoeffDict(JDlist, coeff_dict)
print('HPlist=',HPlist)

# Demonstrate that the HPlist is as long as the JDlist and that each entry is an integer
print('Demonstration: len(JDlist)==len(HPlist)=', len(JDlist)==len(HPlist))
print('Demonstration: np.all([isinstance(_,int) for _ in HPlist])=', np.all([isinstance(_,int) for _ in HPlist]))



	 *** WARNING: RANDOM HEALPIX !!! 
 
HPlist= [457, 86, 2089, 2262, 22, 429, 96, 2698, 2374, 300]
Demonstration: len(JDlist)==len(HPlist)= True
Demonstration: np.all([isinstance(_,int) for _ in HPlist])= True


# --------- Intermediate-Level Data Functions ---------------------
 - 20200122
 - This is basically the "PreCalc" Class 
 - It inherits from the "ObjectCoefficients" Class, which itself inherits from the "Base", "HPData", and "Names" Classes
 - It ... ... ... 
 
     '''
        Primary External Class for accessing MPChecker2's 
        pre-calculated data

        Methods
        -------
        
        upsert()
        
    '''


In [154]:
# Instantiate precalc 
importlib.reload(precalc)
P = precalc.PreCalc() 


In [155]:
# Delete the entire saved-data directory to ensure completely clean ...
shutil.rmtree(P._fetch_data_directory())

In [156]:
# upsert 
'''
    Main method used to insert/update coefficients for an object
    Also handles the healpix calculations used for efficiency-of-read

    Inputs:
    -------
    coeff_dict: dictionary (or iterable of dictionaries)
    - all coefficient data to be saved for an object
    - must contain "name" as a key
    - must contain/enable-calculation-of geocentric-healpix data

    Returns:
    --------
    ????:
    -

'''

# Use upsert to insert a new dictionary of healpix coefficients 
name = demo_name_dict[1944]
new_coeff_dict = demo_coeff_dict[name]
P.upsert(new_coeff_dict)



	 *** WARNING: RANDOM HEALPIX !!! 
 

	 *** WARNING: RANDOM HEALPIX !!! 
 


NameError: name 'HP' is not defined

In [ ]:
# get_nightly_precalcs(JD, HPlist):
'''
    Main method used to query the precalculated healpix
    For a given JD, finds the object-integers for each HP in a list of HPs

    Returns the object-names and coefficient-dictionaries for
    the objects present in each healpix on the specified julian date


    Note that at present *no* attempt is made to ***ALWAYS RETURN NEOs***
    - This needs to be added-to / improved
    - We could (perhaps) return everything closer than 0.X au (on a given JD)

    inputs
    ------
    JD: float or int
     - julian date of the night. If <float> will be silently converted to <int>

    HPlist: list-of-integers 
     - healpix to be queried 
     - if integer (single healpix) supplied, is silently converted to list

    returns
    -------
    dictionary
    - key   = HP
    - value = list of object-integers
'''

# Use some JDs and HP that we expect (from above) to be populated
JD = 2450000
HPlist = [17,18,19,1234] 
P.get_nightly_precalcs(JD, HPlist)
